# Combined framework with Sensitivity

### 0. Introduction

In [52]:
from nmpc_mhe.aux.utils import load_iguess
from nmpc_mhe.pyomo_dae.MHEGen_pyDAE import MheGen_DAE
from sample_mods.cstr_rodrigo.cstr_c_nmpc import cstr_rodrigo_dae
from pyomo.environ import *
import os, sys
import matplotlib.pyplot as plt

### 1. Object set-up

In [53]:
states = ["Ca", "T", "Tj"]
measurements = ['T']
controls = ["u1"]
u_bounds = {"u1": (200, 1000)}
state_bounds = {"Ca": (0.0, None), "T": (2.0E+02, None), "Tj": (2.0E+02, None)}
ref_state = {("Ca", (0,)): 0.010}
mod = cstr_rodrigo_dae(2, 2)  #: Some model

In [54]:
#: MHE-NMPC class
e = MheGen_DAE(mod, 2, states, controls, states, measurements,
               u_bounds=u_bounds,
               ref_state=ref_state,
               override_solver_check=True,
               var_bounds=state_bounds,
               k_aug_executable='/home/dav0/in_dev_/WorkshopFraunHofer/caprese/k_aug/bin/k_aug',
               dot_driver_executable='/home/dav0/in_dev_/WorkshopFraunHofer/caprese/k_aug/dot_sens')

New model at 140363312009536
New timebounds defined!
New model at 140363316764336
New timebounds defined!
Model: unknown Bounds activated
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
New model at 140363312641200
New model at 140363312308128
New timebounds defined!
New timebounds defined!
Model: LSMHE (Least-Squares MHE) Bounds activated


W0[[Initializing MHE]]The Q_MHE and R_MHE matrices are diagonal.


In [55]:
#: Covariance Matrices
Q = {}
U = {}
R = {}
Q['Ca'] = 1.11
Q['T'] = 99.0
Q['Tj'] = 1.1
U['u1'] = 0.22
R['T'] = 1.22
e.set_covariance_disturb(Q)
e.set_covariance_u(U)
e.set_covariance_meas(R)
e.create_rh_sfx()
e.get_state_vars()

{'Ca': [(0,)], 'T': [(0,)], 'Tj': [(0,)]}
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[solve_steady_ref]]labels at 1568343070.--------------------


### 2. Problem loop

In [56]:
#: Initial guesses
e.load_iguess_steady()
load_iguess(e.SteadyRef, e.PlantSample, 0, 0)
e.solve_dyn(e.PlantSample)

-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[Solving with IPOPT	]]SteadyRef.--------------------
/tmp/tmpqudrae0q.pyomo.nl -AMPL 
Ipopt vanilla0: 

List of user-set options:

                                    Name   Value                used
                              bound_push = 1e-07                 yes
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 yes
                                max_iter = 3000                  yes
                       print_info_string = yes                   yes
                      print_user_options = yes                   yes
This is Ipopt version vanilla0, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobia

optimal


                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 yes
                                max_iter = 3000                  yes
                       print_info_string = yes                   yes
                      print_user_options = yes                   yes
This is Ipopt version vanilla0, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:       94
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:       28
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       

optimal


0

In [57]:
#: Prepare MHE
e.init_lsmhe_prep(e.PlantSample)
e.shift_mhe()
e.init_step_mhe()
e.solve_dyn(e.lsmhe,
            skip_update=False,
            max_cpu_time=600,
            ma57_pre_alloc=5, tag="lsmhe")  #: Pre-loaded mhe solve

e.prior_phase()
e.deact_icc_mhe()  #: Remove the initial conditions

-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[init_lsmhe_prep]]Preparation phase MHE.--------------------
fetgt 1 0
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[load_init_state_gen]]Load State to nmpc src_kind=mod.--------------------
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[patch_meas_mhe]]Measurement to:0.--------------------
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[Solving with IPOPT	]]Dummy[MHE].--------------------
/tmp/tmpfzeyscwp.pyomo.nl -AMPL 
Ipopt vanilla0: 

List of user-set options:

                                    Name   Value                used
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 y

optimal


                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 yes
                                max_iter = 3000                  yes
                       print_info_string = yes                   yes
                      print_user_options = yes                   yes
This is Ipopt version vanilla0, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:       94
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:       28
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       

optimal


/tmp/tmpqqp31fkx.pyomo.nl -AMPL 
Ipopt vanilla0: 

List of user-set options:

                                    Name   Value                used
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 yes
                                max_iter = 3000                  yes
                       print_info_string = yes                   yes
                      print_user_options = yes                   yes
This is Ipopt version vanilla0, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:       94
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables.................

...

optimal


-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[solve_dyn]] Model solved to optimality.--------------------
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[load_covariance_prior]]K_AUG w red_hess.--------------------


W[K_AUG]...	[K_AUG_ASL]No n_rhs declared
W[K_AUG]...	[K_AUG_ASL]Target log10mu:= -11.
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zU_out suffix declared, setting zU = 0.
W[K_AUG]...	[K_AUG_ASL]No f_timestamp suffix declared, Fallback to default writing mode.
I[K_AUG]...	[K_AUG_ASL] Filename for dot_sens dot_in_.in
E[K_AUG]...	[MA57_DRIVER]

		Could not fix the accuracy of the problem.
		Try re-writing the problem or give a different point or change "max_refinement_steps"
		Current residual ratio 0.4993; Max residual ratio 1e-10.

W[K_AUG]...	[K_AUG_ASL]Evaluating the inverse of the solution matrix. 
I[K_AUG]...	[DPOTRI_DRIVER] Output file name inv_.in


[K_AUG] written by D.T. @2018

: compute_inv=
compute_inv=
I[K_AUG]...	[K_AUG_ASL]File read succesfull
I[K_AUG]...	[K_AUG_ASL]Number of Right hand sides 0
I[K_AUG]...	[K_AUG_ASL]Number of Right hand sides: 0
I[K_AUG]...	[K_AUG_ASL]Number of variables       : 146
I[K_AUG]...	[K_AUG_ASL]Number of constraints     : 129
I[K_AUG]...	[K_AUG_ASL]Number of valid n_dof     : 17
I[K_AUG]...	[ADJUST_MU]log10(mu) computed=-8.601034229527142 at var_i=0
I[K_AUG]...	[ADJUST_MU]log10(mu) close to the target	-8.601033361066351 at var_i=1
I[K_AUG]...	[FIND_INEQUALITIES]summary: eq: 129, leq: 0, geq: 0 
I[K_AUG]...	[K_AUG_ASL]Nonzeroes in the sparse Jacobian 513
I[K_AUG]...	[GET_HESS_ASL]Objective found
I[K_AUG]...	[GET_HESS_ASL]Nonzeroes in the sparse hessian 67
I[K_AUG]...	[GET_HESS_ASL]Minimization problem detected
I[K_AUG]...	[GET_HESS_ASL]Current objective 0.000000
I[K_AUG]...	[GET_HESS_ASL]Missing nz in the Hessian of the Lag: 94
I[K_AUG]...	[K_AUG_ASL]Barrier term added.
I[K_AUG]...	[K_AUG_ASL]MC1

f_timestamp : Direction=Suffix.EXPORT, Datatype=Suffix.INT
    Key : Value


In [58]:
#: Prepare NMPC
e.find_target_ss()
e.create_nmpc()
e.create_suffixes_nmpc()
e.update_targets_nmpc()
e.compute_QR_nmpc(n=-1)
e.new_weights_olnmpc(1e-04, 1e+06)

W0[[find_target_ss]]No reference state was given, using default.


-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[find_target_ss]]Attempting to find steady state.--------------------
New model at 140363311380432
New timebounds defined!
Model: SteadyRef2 (reference) Bounds activated
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[Solving with IPOPT	]]SteadyRef2 (reference).--------------------
/tmp/tmpgf7te_5j.pyomo.nl -AMPL 
Ipopt vanilla0: 

List of user-set options:

                                    Name   Value                used
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 yes
                                max_iter = 10000                 yes
              

optimal
W0[[Initializing NMPC]]With 5 fe and 3 cp.


-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[solve_dyn]] Model solved to optimality.--------------------
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[find_target_ss]]Target: solve done.--------------------
('Ca', (0,))
target Ca 	key (0,) 	weight 108.769024 	value 0.010000
target u1 	value 364.870073
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
New model at 140363312012336
New timebounds defined!


In [59]:
ca = e.PlantSample.Ca[2, 0]
t = e.PlantSample.T[2, 0]
u = e.PlantSample.u1[2]
tj = e.PlantSample.Tj[2, 0]

ca_mhe = e.lsmhe.Ca[10, 0]
t_mhe = e.lsmhe.T[10, 0]
tj_mhe = e.lsmhe.Tj[10, 0]

spl = []
ul = []
cal = []
tl = []
tjl = []

ca_mhel = []
t_mhel = []
tj_mhel = []

In [ ]:
#: Problem loop
for i in range(0, 100):  #: Five steps
    #: set-point change
    if i in [30 * (j * 2) for j in range(0, 100)]:
        ref_state = {("Ca", (0,)): 0.018}
        e.change_setpoint(ref_state=ref_state, keepsolve=True, wantparams=True, tag="sp")
        e.compute_QR_nmpc(n=-1)
        e.new_weights_olnmpc(1e-04, 1e+06)
    #: set point change
    elif i in [30 * (j * 2 + 1) for j in range(0, 100)]:
        ref_state = {("Ca", (0,)): 0.021}
        e.change_setpoint(ref_state=ref_state, keepsolve=True, wantparams=True, tag="sp")
        e.compute_QR_nmpc(n=-1)
        e.new_weights_olnmpc(1e-04, 1e+06)

    #: Plant
    e.solve_dyn(e.PlantSample, stop_if_nopt=True)
    cal.append(value(ca))
    tl.append(value(t))
    tjl.append(value(tj))
    ul.append(value(u))
    e.update_state_real()  # Update the current state
    e.update_soi_sp_nmpc()  #: To keep track of the state of interest.

    e.update_measurement()  # Update the current measurement
    e.compute_y_offset()  #: Get the offset for y
    #: State-estimation MHE
    e.preparation_phase_mhe(as_strategy=False)
    stat = e.solve_dyn(e.lsmhe,
                       skip_update=False, iter_max=500,
                       jacobian_regularization_value=1e-04,
                       max_cpu_time=600, tag="lsmhe", keepsolve=False, wantparams=False)

    if stat != 0:
        sys.exit()
    #: Prior-phase and arrival cost
    ca_mhel.append(value(ca_mhe))
    t_mhel.append(value(t_mhe))
    tj_mhel.append(value(tj_mhe))
    e.update_state_mhe()  #: get the state from mhe
    e.prior_phase()

    e.print_r_mhe()
    e.print_r_dyn()
    #: Control NMPC
    e.preparation_phase_nmpc(as_strategy=False, make_prediction=False)
    stat_nmpc = e.solve_dyn(e.olnmpc, skip_update=False, max_cpu_time=300, tag="olnmpc")
    if stat_nmpc != 0:
        sys.exit()
    e.print_r_nmpc()
    e.update_u(e.olnmpc)
    #: Plant cycle
    e.cycleSamPlant(plant_step=True)
    e.plant_uinject(e.PlantSample, src_kind="dict", skip_homotopy=True)
    e.noisy_plant_manager(sigma=0.0015, action="apply", update_level=True)

#: print our state of interest

-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I0[[Solving with IPOPT	]]SteadyRef2 (reference).--------------------
Solver log file: '/tmp/tmp5tozuzvh_ipopt.log'
Solver solution file: '/tmp/tmp9vxnvzpl.pyomo.sol'
Solver problem files: ('/tmp/tmp9vxnvzpl.pyomo.nl',)
/tmp/tmp9vxnvzpl.pyomo.nl -AMPL 
Ipopt vanilla0: 

List of user-set options:

                                    Name   Value                used
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 yes
                                max_iter = 500                   yes
                             output_file = sp_log.log            yes
                       print_info_string = yes                   yes
                     

optimal


/tmp/tmpoh42u3vb.pyomo.nl -AMPL 
Ipopt vanilla0: 

List of user-set options:

                                    Name   Value                used
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 1e+06                 yes
                                max_iter = 3000                  yes
                       print_info_string = yes                   yes
                      print_user_options = yes                   yes
This is Ipopt version vanilla0, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:       94
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables.................

optimal


        jacobian_regularization_exponent = 2                     yes
           jacobian_regularization_value = 0.0001                yes
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 300                   yes
                                max_iter = 3000                  yes
                             output_file = init_mhe.txt          yes
                       print_info_string = yes                   yes
                      print_user_options = yes                   yes
This is Ipopt version vanilla0, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:       94
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.......

...

-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I18[[solve_dyn]] Model solved to optimality.--------------------
fetgt 5 4
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I18[[preparation_phase_mhe]]idMHE: Ready.--------------------
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I18[[Solving with IPOPT	]]LSMHE (Least-Squares MHE).--------------------
/tmp/tmp_1l5wqp3.pyomo.nl -AMPL 
Ipopt vanilla0: 

List of user-set options:

                                    Name   Value                used
           jacobian_regularization_value = 0.0001                yes
                           linear_solver = ma57                  yes
                  ma57_automatic_scaling = no                    yes
                          ma57_pre_alloc = 1.5                   yes
                   ma57_small_pivot_flag = 0                     yes
                            max_cpu_time = 600                   yes
                                max_iter = 500

optimal
W[K_AUG]...	[K_AUG_ASL]No n_rhs declared
W[K_AUG]...	[K_AUG_ASL]Target log10mu:= -11.
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zU_out suffix declared, setting zU = 0.
W[K_AUG]...	[K_AUG_ASL]No f_timestamp suffix declared, Fallback to default writing mode.
I[K_AUG]...	[K_AUG_ASL] Filename for dot_sens dot_in_.in
E[K_AUG]...	[MA57_DRIVER]

		Could not fix the accuracy of the problem.
		Try re-writing the problem or give a different point or change "max_refinement_steps"
		Current residual ratio 0.476935; Max residual ratio 1e-10.

W[K_AUG]...	[K_AUG_ASL]Evaluating the inverse of the solution matrix. 
I[K_AUG]...	[DPOTRI_DRIVER] Output file name inv_.in


[K_AUG] written by D.T. @2018

: compute_inv=
compute_inv=
I[K_AUG]...	[K_AUG_ASL]File read succesfull
I[K_AUG]...	[K_AUG_ASL]Number of Right hand sides 0
I[K_AUG]...	[K_AUG_ASL]Number of Right hand sides: 0
I[K_AUG]...	[K_AUG_ASL]Number of variables       : 146
I[K_AUG]...	[K_AUG_ASL]Number of constraints     : 126
I[K_AUG]...	[K_AUG_ASL]Number of valid n_dof     : 20
I[K_AUG]...	[ADJUST_MU]log10(mu) computed=-8.601033752584021 at var_i=0
I[K_AUG]...	[ADJUST_MU]log10(mu) close to the target	-8.601033335029307 at var_i=1
I[K_AUG]...	[FIND_INEQUALITIES]summary: eq: 126, leq: 0, geq: 0 
I[K_AUG]...	[K_AUG_ASL]Nonzeroes in the sparse Jacobian 510
I[K_AUG]...	[GET_HESS_ASL]Objective found
I[K_AUG]...	[GET_HESS_ASL]Nonzeroes in the sparse hessian 73
I[K_AUG]...	[GET_HESS_ASL]Minimization problem detected
I[K_AUG]...	[GET_HESS_ASL]Current objective 0.000038
I[K_AUG]...	[GET_HESS_ASL]Missing nz in the Hessian of the Lag: 91
I[K_AUG]...	[K_AUG_ASL]Barrier term added.
I[K_AUG]...	[K_AUG_ASL]MC1

f_timestamp : Direction=Suffix.EXPORT, Datatype=Suffix.INT
    Key : Value
optimal


-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I18[[solve_dyn]] Model solved to optimality.--------------------
fetgt 5 0
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I18[[load_init_state_gen]]Load State to nmpc src_kind=mod.--------------------
-==--==--==--==--==--==--==--==--==--==--==--==--==--==--==-
I18[[Solving with IPOPT	]]Dummy I 1.--------------------


In [ ]:
err_ca = [abs(ca_mhel[i] - cal[i])/cal[i] for i in range(0, len(cal))]
err_t = [abs(t_mhel[i] - tl[i])/tl[i] for i in range(0, len(tl))]
err_tj = [abs(tj_mhel[i] - tjl[i])/tjl[i] for i in range(0, len(tjl))]

In [ ]:
ax1 = plt.subplot(3, 1, 1)
plt.plot(err_ca)
plt.ylabel("$C_{a}$ error")
plt.axvline(x=30, color="r")
plt.axvline(x=90, color="r")


plt.subplot(3, 1, 2, sharex=ax1)
plt.plot(err_t)
plt.ylabel("$T$ error")
plt.axvline(x=30, color="r")
plt.axvline(x=90, color="r")


plt.subplot(3, 1, 3, sharex=ax1)
plt.plot(err_tj)
plt.ylabel("$T_{j} error$")
plt.axvline(x=30, color="r")
plt.axvline(x=90, color="r")
plt.show()

In [ ]:
ax1 = plt.subplot(3, 1, 1)
plt.plot(cal)
plt.plot(spl)
plt.ylabel("$C_{a}$")
plt.axvline(x=30, color="r")
plt.axvline(x=90, color="r")
plt.legend(["NMPC", "Set-point"])

plt.subplot(3, 1, 2, sharex=ax1)
plt.plot(tl)
plt.axvline(x=30, color="r")
plt.axvline(x=90, color="r")
plt.ylabel("$T$")


plt.subplot(3, 1, 3, sharex=ax1)
plt.plot(ul)
plt.ylabel("$u$")
plt.axvline(x=30, color="r")
plt.axvline(x=90, color="r")
#plt.title("Control input")
plt.show()

### Credits:
 - David Thierry